## Stationarity type for each series 

### *Very* small difference in ACF score between additive score and multiplicative score
- Except for OCC which has a strong preference for multiplicative
- To make it simple we can do ALL regions in multiplicative

### Multiplicative
- nights_IDF: Jinny
- nights_NAQ: Jinny
- nights_ARA: Jinny
- nights_PAC: Jinny
- nights_CVL: Jinny


- nights_BFC: Sekou
- nights_NOR: Sekou
- nights_HDF: Sekou
- nights_GES: Sekou


- nights_PDL: Franck
- nights_BRE: Franck
- nights_OCC: Franck
- nights_COR: Franck

#### See calculations below if needed

In [1]:
import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
data_nights = pd.read_csv('/Users/jinny/Documents/touristcast/datasets/nights_2010-2017.csv', index_col='date')
data_nights = data_nights.drop('Unnamed: 0', axis=1)
data_nights.index = pd.to_datetime(data_nights.index)
data_nights.columns

Index(['nights_IDF', 'nights_CVL', 'nights_BFC', 'nights_NOR', 'nights_HDF',
       'nights_GES', 'nights_PDL', 'nights_BRE', 'nights_NAQ', 'nights_OCC',
       'nights_ARA', 'nights_PAC', 'nights_COR'],
      dtype='object')

### NONE of the nights are stationary so we will need to decompose each time

In [3]:
from statsmodels.tsa.stattools import adfuller

for column in data_nights.columns:
    result = adfuller(data_nights[column])
#     print(f'Stationarity test for variable {column}')
#     print('ADF Statistic: %f' % result[0])
#     print('p-value: %f' % result[1])
#     print('Critical Values:')

#     for key, value in result[4].items():
#         print('\t%s: %.3f' % (key, value))
    
    if result[1] < 0.05:
        print(f'{column}')
        print('Stationary')
        print('-------')

### Stationarity type for each Nights series

In [4]:
from statsmodels.tsa.seasonal import seasonal_decompose

# ----- Additive Model below -----

add_data = seasonal_decompose(data_nights, model='additive')  

add_trend = add_data.trend.dropna()
add_seasonal = add_data.seasonal.dropna()
add_residual = add_data.resid.dropna()

# stationarity_test(add_residual)

# ----- Multiplicative Model below -----

mult_data = seasonal_decompose(data_nights, model='multiplicative')  

mult_trend = mult_data.trend.dropna()
mult_seasonal = mult_data.seasonal.dropna()
mult_residual = mult_data.resid.dropna()

# stationarity_test(mult_residual)

In [18]:
from statsmodels.tsa.stattools import acf

add_acfs = add_residual.apply(acf, axis=0, fft=False).apply(np.square, axis=0).apply(np.sum)

mult_acfs = mult_residual.apply(acf, axis=0, fft=False).apply(np.square, axis=0).apply(np.sum)

additive = add_acfs < mult_acfs
add_cols = additive.index[additive].tolist()
mult_cols = additive.index[~additive].tolist()
print('Additive:', add_cols)
print('Multiplicative:', mult_cols)
print('------')
print('Positive = Multiplicative, Negative = Additive', '\n', add_acfs - mult_acfs)

Additive: ['nights_IDF', 'nights_NAQ', 'nights_ARA', 'nights_PAC']
Multiplicative: ['nights_CVL', 'nights_BFC', 'nights_NOR', 'nights_HDF', 'nights_GES', 'nights_PDL', 'nights_BRE', 'nights_OCC', 'nights_COR']
------
Positive = Multiplicative, Negative = Additive 
 nights_IDF   -0.034117
nights_CVL    0.007227
nights_BFC    0.187358
nights_NOR    0.114303
nights_HDF    0.000648
nights_GES    0.063095
nights_PDL    0.073486
nights_BRE    0.035301
nights_NAQ   -0.058864
nights_OCC    0.424805
nights_ARA   -0.049186
nights_PAC   -0.000010
nights_COR    0.107867
dtype: float64
